In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from pandas import DataFrame
from numpy import mean
from bs4 import BeautifulSoup
import re
from math import ceil
from time import sleep

In [3]:
def PubMed_crawler(target,button1 = True,button2 = False,button3 = False,search_year_range = '2010-2021'): 
    '''
    target : PubMed上的輸入
    buttons : PubMed左側的選單
    search_year_range : 年份範圍
    '''
    # Setting #
#     button1 = True;button2 = False;button3 = False # PubMed左側的選單
#     search_year_range = '2010-2021'
    # Setting #  
    
    # Ready #
    html_paper = "https://pubmed.ncbi.nlm.nih.gov/"
    html = html_paper + '?term='
    chrome_options = Options()
    chrome_options.add_argument('--headless')#無介面操作
    driver,driver2,driver3,driver4 = webdriver.Chrome(chrome_options=chrome_options),webdriver.Chrome(chrome_options=chrome_options),webdriver.Chrome(chrome_options=chrome_options),webdriver.Chrome(chrome_options=chrome_options)

    mode = ''
    if(button1==True): mode += '&filter=simsearch1.fha'
    if(button2==True): mode += '&filter=simsearch2.ffrft'
    if(button3==True): mode += '&filter=simsearch3.fft'
    year = ''
    year += '&filter=years.' + search_year_range
    # Ready #
    
    driver.get(html+target+mode+year)
    result = driver.find_element_by_xpath('//*[@id="search-results"]/div[2]/div[1]').text
    want_papers = int(input('共 '+result+' 個結果，目標前K篇'))
    show_times = ceil(want_papers/10)+1
    mode += '&page='
    
    # Start #
    
    ID_result,ID_paper_year_fulls,ID_papers_titles,ID_Abstracts,ID_cited_bys,ID_Journal_names,ID_IF = [],[],[],[],[],[],[]
    Journal_appear,All_papers_Journal,All_Final_collects,Search_answer,collect_search,collect_search_len,Search_Answer = [],[],[],[],[],[],[]
    PMCID_years,Gene_types,Disease_types,Chemical_types,Mutation_types,Species_types,CellLine_types = [],[],[],[],[],[],[]
    want_papers_need = want_papers
    IDs = 0 ; sleep_sec = 2 ; Journal_Title_count = 0

    driver3.get("http://2020if.home-for-researchers.com/static/index.html#/")

    for page in range(1,show_times): # if stop could restart
        driver.get(html+target+mode+str(page)) # each page

        for each_id in range(1,11): # each ID
            if(page==1) : ID_paper = driver.find_element_by_xpath('//*[@id="search-results"]/section/div['+str(page)+']/div/article['+str(each_id)+']/div[2]/div[1]/div[1]/span[5]').text
            else : ID_paper = driver.find_element_by_xpath('//*[@id="search-results"]/section/div[2]/div/article['+str(each_id)+']/div[2]/div[1]/div[1]/span[5]').text
            ID_result.append( ID_paper[6:] ) # get ID

            driver2.get(html_paper+ID_paper[6:]+'/')
            Gene_type,Disease_type,Chemical_type,Mutation_type,Species_type,CellLine_type = [],[],[],[],[],[]
            try:
                Create_save_list = []
                Create_save_list.append(driver2.find_element_by_xpath('//*[@id="full-view-heading"]').text) # ID_paper_year_full
                Create_save_list.append(driver2.find_element_by_xpath('//*[@id="full-view-heading"]/h1').text) # ID_paper_title
                Create_save_list.append(driver2.find_element_by_xpath('//*[@id="enc-abstract"]').text) # ID_Abstract
            except: # 防呆
                driver2.get(html_paper+ID_paper[6:]+'/')
                sleep(sleep_sec)
                Create_save_list = []
                for slow_down_speed in [ '//*[@id="full-view-heading"]' , '//*[@id="full-view-heading"]/h1' , '//*[@id="enc-abstract"]' ]:
                    try:
                        Create_save_list.append(driver2.find_element_by_xpath(slow_down_speed).text)
                    except:
                        Create_save_list.append('')
            try:
                Create_save_list.append(driver2.find_element_by_xpath('//*[@id="citedby"]/h2/em[1]').text) # ID_cited_by
            except:
                Create_save_list.append('0')

            each_PMID_list = str(ID_paper[6:])
            driver4.get('https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocxml?pmids='+str(each_PMID_list)+'&concepts=')
            soup = BeautifulSoup(driver4.page_source, 'html.parser')
            try:
                string = ''
                passages = soup.find_all('passage')
                for passage in passages :
                    annotations = passage.find_all('annotation')
                    for annotation in annotations :
                        NE = annotation.find('text')
                        Type = annotation.find('infon', {'key': re.compile("type")})
                        if Type.text.lower() == 'gene' :
                            Gene_type.append(NE.text)
                        elif Type.text.lower() == 'disease' :
                            Disease_type.append(NE.text)
                        elif Type.text.lower() == 'chemical' :
                            Chemical_type.append(NE.text)
                        elif Type.text.lower() == 'mutation' :
                            Mutation_type.append(NE.text)
                        elif Type.text.lower() == 'species' :
                            Species_type.append(NE.text)
                        elif Type.text.lower() == 'cellline' :
                            CellLine_type.append(NE.text)
                    if annotations == [] :
                        string += '\n'
                    else:
                        string += '\n\n'
                year = soup.find('infon', {'key': re.compile("year")})
                year_text = year.text
                PMCID_years.append(year_text)
            except:
                No_ = True

            Gene_types.append(Gene_type)
            Disease_types.append(Disease_type)
            Chemical_types.append(Chemical_type)
            Mutation_types.append(Mutation_type)
            Species_types.append(Species_type)
            CellLine_types.append(CellLine_type)

            # -------------------- Pubtator-----------------------

            ID_paper_year_fulls.append(Create_save_list[0])
            ID_papers_titles.append(Create_save_list[1])
            ID_Abstracts.append(Create_save_list[2])
            ID_cited_bys.append(Create_save_list[3])

            # IF #
            driver2.find_element_by_id("full-view-journal-trigger").click() # 點 Journal 連結
            sleep(sleep_sec)
            driver2.find_element_by_link_text("Search in NLM Catalog").click()
            try : # 防止有多個可能 Journal 干擾
                Journal_Title = driver2.find_element_by_xpath('//*[@id="maincontent"]/div/div[5]/div/h1').text 
                ID_Journal_names.append(Journal_Title)
                if Journal_Title not in Journal_appear :
                    Journal_detail = []
                    Journal_appear.append( Journal_Title )
                    Detail = driver2.find_element_by_xpath('//*[@id="maincontent"]/div/div[5]/div/div[1]').text
                    Detail_arr = Detail.split('\n')

                    Index,Data,data = [],[],[]
                    Next_start,Next_end = False,False

                    for each_word in Detail_arr :
                        if each_word[-1] == ':' :
                            Index.append(each_word)
                            if Next_start :
                                Next_end = True
                                Data.append(data)
                                data = []
                        else:
                            Next_start = True
                            data.append(each_word)
                    Data.append(data)
                    Journal_data = DataFrame(Data,index=Index)
                    All_papers_Journal.append(Journal_data)
                    Target_index = Journal_data.index

                    Loc,Final_collects = [],[]
                    for title_index in Target_index :
                        if title_index.find('Title') != -1 :
                            Loc.append(title_index)    
                    Collects = Journal_data.loc[Loc].values
                    for Collects_ in Collects :
                        for collect in Collects_ :
                            if str(collect) != 'None' :
                                Final_collects.append(collect)
                    Journal_Title_count += 1

                    # Impact factor real get
                    Search_answer = []
                    collect_search = []
                    collect_search_len = []
                    for search in Final_collects :
                        if str(search) != 'None' :
                            collect_search.append(search)
                            collect_search_len.append(len(search))
                    collect_search_len_sorted, collect_search_sorted = (list(t) for t in zip(*sorted(zip(collect_search_len, collect_search))))
                    collect_search_len_sorted = collect_search_len_sorted[::-1]
                    collect_search_sorted = collect_search_sorted[::-1]
                    IF = ''
                    Level = 'X'
                    temporarily = ''
                    temporarily_raw = ''
                    IF_temporarily = ''
                    for show in collect_search_sorted :
                        if show.find(':') == -1 and show.find('(') == -1 and show.find('=') == -1 and show.find(';') == -1:
                            show_ = True
                        else:
                            if show.find(' :') != -1 :
                                show = show[:show.find(' :')]
                            if show.find(' (') != -1 :
                                show = show[:show.find(' (')]
                            if show.find(' =') != -1 :
                                show = show[:show.find(' =')]
                            if show.find(';') != -1 :
                                show = show[:show.find(';')]
                        driver3.find_element_by_xpath("//input[@id='']").clear()
                        driver3.find_element_by_xpath("//input[@id='']").send_keys(show)
                        driver3.find_element_by_xpath("//div[@id='app']/div[3]/div[2]/div/div/div[2]/div/div[2]/span[2]/span").click()
                        sleep(sleep_sec)

                        try:
                            IF_web_name = driver3.find_element_by_xpath('//*[@id="app"]/div[3]/div[2]/div/div[2]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div[2]').text
                            if temporarily_raw == '':
                                temporarily_raw = IF_web_name
                            if IF_web_name[:4] == '(水刊)' :
                                IF_web_name = IF_web_name[4:]

                            for ch in '()!"#$&+/:;<=>?@[\\]^_{|}·~‘⦁,.: #- ’': #- ’ 
                                show = show.replace(ch,"")
                                IF_web_name = IF_web_name.replace(ch,"")

                            IF_web_name2 = ''
                            if IF_web_name[:4] == 'The ' :
                                IF_web_name2 = IF_web_name[4:]

                            if temporarily == '':
                                temporarily = IF_web_name
                            if show.lower() == IF_web_name.lower() or show.lower() == IF_web_name2.lower():
                                IF = driver3.find_element_by_xpath('//*[@id="app"]/div[3]/div[2]/div/div[2]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div[3]/span[2]/span').text
                                Level = '1'
                                search_N = IF_web_name
                                search_N_raw = driver3.find_element_by_xpath('//*[@id="app"]/div[3]/div[2]/div/div[2]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div[2]').text
                                break
                            elif IF_web_name.lower().find(show.lower())!= -1 or show.lower().find(IF_web_name.lower()) != -1:
                                search_N_raw = driver3.find_element_by_xpath('//*[@id="app"]/div[3]/div[2]/div/div[2]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div[2]').text
                                search_N = IF_web_name
                                IF = driver3.find_element_by_xpath('//*[@id="app"]/div[3]/div[2]/div/div[2]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div[3]/span[2]/span').text
                                Level = '2'
                                break
                            else:
                                if IF_temporarily == '':
                                    IF_temporarily = driver3.find_element_by_xpath('//*[@id="app"]/div[3]/div[2]/div/div[2]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div[3]/span[2]/span').text
                        except:
                            Not_found = True
        #                 print('Level '+Level+' : ',IF)
                    if Level == 'X' :
                        search_N = temporarily
                        search_N_raw = temporarily_raw
                        IF = IF_temporarily
                    Search_answer.append(Level)
                    Search_answer.append(search_N)
                    Search_answer.append(search_N_raw)
                    Search_answer.append(IF)
                    Search_Answer.append(Search_answer)
                    ID_IF.append(IF)
                else:
                    journal_loc_2_if = Journal_appear.index(Journal_Title)
                    ID_IF.append(Search_Answer[journal_loc_2_if][3])

            except:
                Maybe_more_than_one = True
                ID_Journal_names.append('')
                ID_IF.append(None)
            # IF #

            if mean([len(ID_paper_year_fulls),len(ID_Abstracts),len(ID_papers_titles),len(ID_cited_bys),len(ID_Journal_names),len(ID_IF)]) != len(ID_paper_year_fulls) :
                print(IDs,end=' ')
                break
            IDs += 1
            print('('+str(IDs)+')',end=' ')
            want_papers_need -= 1
            if(want_papers_need==0) : break
        if(want_papers_need==0) : break
            
    # Start #
    
    IF_status = DataFrame(Search_Answer,index=Journal_appear,columns=['Level','Journal name','Journal name raw','IF'])
    PMID_order = [ 'order'+str(order_+1) for order_ in range(len(ID_result)) ]
    PubMed_crawler_output = DataFrame([ID_result,ID_paper_year_fulls,ID_papers_titles,ID_Abstracts,ID_Journal_names,ID_IF,ID_cited_bys,PMCID_years,Gene_types,Disease_types,Chemical_types,Mutation_types,Species_types,CellLine_types],index=['PMID','Paper attributes','Title','Abstract','Journal','IF','Cited by','Year','Gene_types','Disease_types','Chemical_types','Mutation_types','Species_types','CellLine_types'],columns=PMID_order).T

    driver.quit()
    driver2.quit()
    driver3.quit()
    driver4.quit()
    
    return IF_status,PubMed_crawler_output

In [4]:
IF_status,PubMed_crawler_output = PubMed_crawler('colorectal cancer AND Bacteria')

C:\Users\Tingchun.TC.Hung\Anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


KeyboardInterrupt: Interrupted by user

In [4]:
PubMed_crawler_output

,PMID,Paper attributes,Title,Abstract,Journal,IF,Cited by,Year,Gene_types,Disease_types,Chemical_types,Mutation_types,Species_types,CellLine_types
order1,31450675,Review\nInt J Mol Sci\n. 2019 Aug 25;20(17):41...,Oral Bacteria and Intestinal Dysbiosis in Colo...,The human organism coexists with its microbiot...,International journal of molecular sciences,4.556,29,2019,[],"[Dysbiosis in Colorectal Cancer, periodontitis...",[],[],"[human, Porphyromonas gingivalis]",[]
order2,28106826,Review\nInt J Mol Sci\n. 2017 Jan 19;18(1):197...,Colorectal Carcinoma: A General Overview and F...,Colorectal cancer (CRC) is the third most comm...,International journal of molecular sciences,4.556,206,2017,"[TP53, MYC, KRAS, BRAF, PIK3CA, PTEN, SMAD2, S...","[Colorectal Carcinoma, Colorectal Cancer, Colo...",[],[],"[Bacteroides fragilis, Escherichia coli, patient]",[]
order3,26811603,Review\nWorld J Gastroenterol\n. 2016 Jan 14;2...,Gut microbiota imbalance and colorectal cancer,The gut microbiota acts as a real organ. The s...,World journal of gastroenterology,3.665,191,2016,[],"[colorectal cancer, infection, inflammatory bo...",[],[],"[Streptococcus bovis, Helicobacter pylori, Bac...",[]
order4,31554963,Review\nNat Rev Gastroenterol Hepatol\n. 2019 ...,Gut microbiota in colorectal cancer: mechanism...,Colorectal cancer (CRC) accounts for about 10%...,Nature reviews. Gastroenterology & hepatology,29.848,114,2019,[],"[colorectal cancer, Colorectal cancer, CRC, ca...",[],[],"[patients, Fusobacterium nucleatum, Escherichi...",[]
order5,28632155,Review\nInt J Mol Sci\n. 2017 Jun 20;18(6):131...,"Microbiota, Inflammation and Colorectal Cancer","Colorectal cancer, the fourth leading cause of...",International journal of molecular sciences,4.556,58,2017,[],"[Inflammation, Colorectal Cancer, Colorectal c...",[],[],[patients],[]
order6,28063002,Review\nEur J Clin Microbiol Infect Dis\n. 201...,Gut microbiota and colorectal cancer,The gut microbiota is considered as a forgotte...,European journal of clinical microbiology & in...,2.837,53,2017,[],"[colorectal cancer, gastrointestinal diseases,...",[],[],[human],[]
order7,31586566,Review\nGastroenterology\n. 2020 Jan;158(2):32...,"Influence of the Gut Microbiome, Diet, and Env...",Researchers have discovered associations betwe...,Gastroenterology,17.373,67,2019,[],"[Colorectal Cancer, colorectal cancer, CRC, CR...",[],[],[Gut Microbiome],[]
order8,25198138,Review\nNat Rev Microbiol\n. 2014 Oct;12(10):6...,"The gut microbiota, bacterial metabolites and ...",Accumulating evidence suggests that the human ...,Nature reviews. Microbiology,34.209,630,2014,[],"[colorectal cancer, colorectal cancer, inflamm...","[short-chain fatty acids, acetate, propionate,...",[],[human],[]
order9,33329610,Review\nFront Immunol\n. 2020 Nov 30;11:615056...,The Intestinal Microbiota and Colorectal Cancer,"The intestinal microbiota, composed of a large...",Frontiers in immunology,5.085,15,2020,[],"[Colorectal Cancer, dysbiosis, colorectal canc...",[],[],"[human, Streptococcus bovis, Bacteroides fragi...",[]
order10,28753429,Cell\n. 2017 Jul 27;170(3):548-563.e16. doi: 1...,Fusobacterium nucleatum Promotes Chemoresistan...,Gut microbiota are linked to chronic inflammat...,Cell,38.637,397,2017,[],"[Colorectal Cancer, chronic inflammation, carc...",[],[],"[Fusobacterium nucleatum, patients, patients, ...",[]


In [5]:
IF_status

,Level,Journal name,Journal name raw,IF
International journal of molecular sciences,1,INTERNATIONALJOURNALOFMOLECULARSCIENCES,INTERNATIONAL JOURNAL OF MOLECULAR SCIENCES,4.556
World journal of gastroenterology,1,WORLDJOURNALOFGASTROENTEROLOGY,(水刊)WORLD JOURNAL OF GASTROENTEROLOGY,3.665
Nature reviews. Gastroenterology & hepatology,1,NatureReviewsGastroenterologyandHepatology,Nature Reviews Gastroenterology and Hepatology,29.848
European journal of clinical microbiology & infectious diseases : official publication of the European Society of Clinical Microbiology,X,EUROPEANJOURNALOFCLINICALMICROBIOLOGYandINFECT...,EUROPEAN JOURNAL OF CLINICAL MICROBIOLOGY and ...,2.837
Gastroenterology,1,GASTROENTEROLOGY,GASTROENTEROLOGY,17.373
Nature reviews. Microbiology,1,NATUREREVIEWSMICROBIOLOGY,NATURE REVIEWS MICROBIOLOGY,34.209
Frontiers in immunology,1,FrontiersinImmunology,Frontiers in Immunology,5.085
Cell,1,CELL,CELL,38.637
